In [1]:
import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning)

In [2]:
import numpy as np
import pandas as pd
import gc

/home/ramezani/anaconda/lib/python2.7/site-packages/pandas/core/computation/__init__.py:18: UserWarning: The installed version of numexpr 2.4.3 is not supported in pandas and will be not be used
The minimum supported version is 2.4.6

  ver=ver, min_ver=_MIN_NUMEXPR_VERSION), UserWarning)


In [3]:
path = "../../data/"
print("Read the properties and merge with coord")
coord = pd.read_csv(path + 'renamed_properties_2016.csv', usecols = ["id_parcel", "latitude", "longitude"])
print "Size of the properties data frame: ", coord.shape

Read the properties and merge with coord
Size of the properties data frame:  (2985217, 3)


In [4]:
index = pd.isnull(coord["latitude"])
coord = coord[~index]
print coord.shape

(2973780, 3)


In [5]:
coord['latitude'] = coord['latitude'] / 1e6
coord['longitude'] = coord['longitude'] / 1e6

In [6]:
mid_lat = coord['latitude'].median()
mid_lon = coord['longitude'].median()

import math
# Functions to convert the coordinates to polar system and to rotate the coordinate system
def cart2rho(x, y):
    rho = np.sqrt(x**2 + y**2)
    return rho

def cart2phi(x, y):
    phi = np.arctan2(y, x)
    return phi

def rotation_x(row, alpha):
    x = row['latitude']
    y = row['longitude']
    return x*math.cos(alpha) + y*math.sin(alpha)

def rotation_y(row, alpha):
    x = row['latitude']
    y = row['longitude']
    return y*math.cos(alpha) - x*math.sin(alpha)

def add_rotation(degrees, df):
    namex = "rot" + str(degrees) + "_X"
    namey = "rot" + str(degrees) + "_Y"

    df['num_' + namex] = df.apply(lambda row: rotation_x(row, math.pi/(180/degrees)), axis=1)
    df['num_' + namey] = df.apply(lambda row: rotation_y(row, math.pi/(180/degrees)), axis=1)

    return df

def operate_on_coordinates(df):
    for df in [df]:
        #polar coordinates system
        df["num_rho"] = df.apply(lambda x: cart2rho(x["latitude"] - mid_lat, x["longitude"]+mid_lon), axis=1)
        df["num_phi"] = df.apply(lambda x: cart2phi(x["latitude"] - mid_lat, x["longitude"]+mid_lon), axis=1)
        #rotations
        for angle in [15,30,45,60,75]:
            df = add_rotation(angle, df)

    return df

coord = operate_on_coordinates(coord)

KeyboardInterrupt: 

In [11]:
coord.to_csv(path + "location2.csv", index=False)